# Finding Steady states

Searching steady states in RTG4 model `rtgM4`.

Use `SSRootfind` to find both stable and unstable nodes.

In [ ]:
import RetroSignalModel as rs
using DifferentialEquations
# Construct the model
rtgM4 = rs.rtgM4(1)

In [ ]:
# Reaction system
rn = rtgM4.model

In [ ]:
# System parameters
params = rtgM4.p

Reference initial conditions optimized by Jump and expression data

In [ ]:
# Reference initial conditions optimized by Jump and expression data
u0 = rtgM4.u
statemap = Dict(k => i for (i,) in enumerate(keys(u0)))

Calculate conservation relationships

In [ ]:
Σbmh = u0.Bmh + u0.BmhMks
ΣMks = u0.Mks + u0.BmhMks + u0.Rtg2Mks_c
ΣRtg1 = u0.Rtg13_a_c + u0.Rtg13_i_c + u0.Rtg1_c + u0.Rtg1_n + u0.Rtg13_a_n + u0.Rtg13_i_n
ΣRtg2 = u0.Rtg2Mks_c + u0.Rtg2_ina_c + u0.Rtg2_act_c
ΣRtg3 = u0.Rtg13_a_c + u0.Rtg13_i_c + u0.Rtg13_a_n + u0.Rtg13_i_n + u0.Rtg3_i_c + u0.Rtg3_a_c + u0.Rtg3_a_n + u0.Rtg3_i_n

In [ ]:
"""
Create a set of initial conditions respecting conservation relationships for the ensemble

    rand_func: a function that returns a valeu between 0 and 1
"""
function make_u0!(u0, Σbmh, ΣMks, ΣRtg1, ΣRtg2, ΣRtg3; rand_func=rand)

    # Randomly make n fractions of total
    function _frac(n::Int, total=1)
        total .* diff([0.0; sort([rand_func() for _ in 1:(n-1)]); 1.0])
    end

    # Split ΣMks into 3
    frac = _frac(3, ΣMks)
    u0[statemap[:Mks]] = frac[1]
    u0[statemap[:BmhMks]] = frac[2]
    u0[statemap[:Rtg2Mks_c]] = frac[3]
    u0[statemap[:Bmh]] = Σbmh - u0[statemap[:BmhMks]]

    remainder = ΣRtg2 - u0[statemap[:Rtg2Mks_c]]
    u0[statemap[:Rtg2_ina_c]] = remainder * rand_func()
    u0[statemap[:Rtg2_act_c]] = remainder - u0[statemap[:Rtg2_ina_c]]

    # Split ΣRtg1 into 6
    frac = _frac(6, ΣRtg1)

    u0[statemap[:Rtg13_a_c]] = frac[1]
    u0[statemap[:Rtg13_i_c]] = frac[2]
    u0[statemap[:Rtg1_c]] = frac[3]
    u0[statemap[:Rtg1_n]] = frac[4]
    u0[statemap[:Rtg13_a_n]] = frac[5]
    u0[statemap[:Rtg13_i_n]] = frac[6]

    # Split ΣRtg3 into 4
    remainder = ΣRtg3 - (u0[statemap[:Rtg13_a_c]] + u0[statemap[:Rtg13_i_c]] + u0[statemap[:Rtg13_a_n]] + u0[statemap[:Rtg13_i_n]])
    frac = _frac(4, remainder)

    u0[statemap[:Rtg3_i_c]] = frac[1]
    u0[statemap[:Rtg3_a_c]] = frac[2]
    u0[statemap[:Rtg3_a_n]] = frac[3]
    u0[statemap[:Rtg3_i_n]] = frac[4]

    return u0
end

function prob_func(prob, i, repeat)
    make_u0!(prob.u0, Σbmh, ΣMks, ΣRtg1, ΣRtg2, ΣRtg3)
    prob
end

In [ ]:
"""Reject invalid (NaNs and negatives) and duplicate results"""
function reduction(u, batch, I; negtol=-1e-6)
    for result in batch
        # Skip invalid (NaNs and negatives) results
        if any(isnan.(result)) || any(result .< negtol)
            continue
        end

        result .= max.(0, result)
        isUnique = true

        # Only save unique solutions
        for existing in u
            if all(isapprox.(existing, result, rtol=1e-4))
                isUnique = false
                break
            end
        end

        if isUnique
            u = push!(u, result)
        end
    end
    return (u, false)
end

In [ ]:
## Ensemble simulation
oprob = ODEProblem(rn, u0, (0.0, 1000.0), params)
ssprob = SteadyStateProblem(oprob)
method = SSRootfind()
ensprob = EnsembleProblem(ssprob, prob_func=prob_func, reduction=reduction)

In [ ]:
trajectories = 120000
batch_size = 300
@time sim = solve(ensprob, method, EnsembleThreads(); trajectories=trajectories, batch_size=batch_size)

**TODO**

Steady states at different levels of input signal `S`. (Use a loop).

## Saving results

> WIP

In [ ]:
using Parameters
using CSV
using DataFrames
using Random
using SteadyStateDiffEq
using LaTeXStrings
using SteadyStates: merge_csv, create_null_dataframe, find_undef_name, valid

In [ ]:
savedir =  joinpath("SteadyStates/data/steady_states_")

In [ ]:
# For Saving to Datafreme. 
if false

    ks = keys(param.u)
    filename = joinpath(save_dir, "steady_states_PLACEHOLDER.csv")

    # number of iteration
    ind = Int(64800)
    iter_number = 10

    for i in ProgressBar(1:iter_number)

        df = create_null_dataframe(ks) # renew df. Clear the memory

        param_ranges = [  
            0.:1.,
            100.:1000.,
            0.:100.,
            0.:1.,
            0.:10.,
            100.:10000.,
            0.:1000.,
            0.:10.,
            0.:100.,
            0.:1000.,
            0.:100.,
            0.:100.,
            0.:10.,
            0.:100.,
            0.:100.,
            0.:100.,
            0.:10.,
        ]

        pr = ParameterRandom(
            param_ranges,
            len = ind)

        sols = solve(de, pr);


        for u in sols.u
            any(isnan, u) ? continue : nothing # skip NaN value
            d = Dict(keys(param.u) .=> u)
            push!(df,d)
        end



        csvname = find_undef_name(filename, "PLACEHOLDER")

        nrow(df) == 0 ? nothing : CSV.write(csvname, df) # skip null dataframe

    end
end 

## Merge files

In [ ]:
df = merge_csv(savedir);

df_ = valid(df; low=0.,high=1e4, s_low=0., s_high=1.0) # valid dataset
sort!(df_)
CSV.write(joinpath("SteadyStates/data", "valid_ss.csv"),df_)

In [ ]:
df_ = valid(df; low=0.,high=1e4, s_low=0., s_high=1.0) # valid dataset

## Plot Steady States Results in Different Conditions

In [ ]:
using Pkg 
Pkg.activate("SteadyStates")
Pkg.instantiate()
using IJulia
using TikzPictures


In [ ]:
tp = TikzPicture("\\draw (0,0) -- (10,10);\n\\draw (10,0) -- (0,10);\n\\node at (5,5) {tikz \$\\sqrt{\\pi}\$};", options="scale=0.25", preamble="")
save(PDF("test"), tp)

In [ ]:

"""
Try conditions of the model with given initial variables and parameters.
"""
function Try_conditions(u,p, VALID_SETTING;
                        BREAK=false,
                        Cond_Random=false)

        @unpack  model, S_SPAN, DEL_CONC, TRANS_THRESHOLD,  DEFAULT_SSMETHOD, cond, protein_lookup = VALID_SETTING

        valid = 1

        num_cond = size(cond)[1] # number of conditions

        # Shuffling the conditions
        trial_cond_order = Cond_Random ? shuffle(1:num_cond) : 1:num_cond

        test_log = Dict("nuc"=>[], "cyt"=>[])

        # get initial condition
        de = DEsteady(func=model, u0=u, p=p, method=DEFAULT_SSMETHOD)
        #u_ = solve(de) # This step changed the initial value, and reset as the steady ones
        u_ = redist2monomer(u, protein_lookup)

        for trial_n in trial_cond_order # Test conditions

            con = cond[trial_n,:]

            if typeof(con.Trans2Nuc) == Missing 
                push!(test_log["nuc"] , Missing)
                push!(test_log["cyt"], Missing)
                continue  # some conditions are not measured
            end
            

            ud = Get_del_u(u_, con.rtg1, con.rtg2, con.rtg3, con.mks, con.s, protein_lookup; del_conc=DEL_CONC, s_span=S_SPAN)

            sol = solve(de(ud))

            # Check the steady-state is real
            if sol.retcode==:Failure
                valid = 0 #unstable system
                break
            end


            trans2nuc = check_nu_accumulation(sol, con.gfp, protein_lookup, threshold=TRANS_THRESHOLD) # 1 means nucleus has significantly higer concentration than cytosol

            
  
            push!(test_log["nuc"] , trans2nuc[1])
            push!(test_log["cyt"], trans2nuc[2])
            
 
    end
    
        return test_log
end


function check_nu_accumulation(sol, gfp, protein_lookup; threshold = TRANS_THRESHOLD)

    if gfp == "rtg1"
        cyt_index = protein_lookup[:Rtg1_c]
        nuc_index = protein_lookup[:Rtg1_n]
    elseif gfp == "rtg3"
        cyt_index = protein_lookup[:Rtg3_c]
        nuc_index = protein_lookup[:Rtg3_n]
    else
        throw(MathodError)
    end

    total_conc_cyt = sum(sol[cyt_index])
    total_conc_nuc = sum(sol[nuc_index])

  
    return (total_conc_nuc, total_conc_cyt)
end

function Get_del_u(u, rtg1, rtg2, rtg3, mks, s, protein_lookup; del_conc=DEL_CONC, s_span=S_SPAN )

    del_info = Dict([:Rtg1, :Rtg2, :Rtg3, :Mks] .=> [rtg1, rtg2, rtg3, mks])
    ud = deepcopy(u)

    for protein in keys(del_info)
        protein_existence = del_info[protein]
        if protein_existence == 0
            ud[protein_lookup[protein]] .= del_conc
        end
    end

    ud[protein_lookup[:s]] = s_span[s+1] #in boolean, s=0 means 1 in span_tuple

    return ud
end


function redist2monomer(u, protein_lookup)
    u_dist = zeros(length(u))
    
    
    # Rtg2 
    sum_rtg2 = sum(u[protein_lookup[:Rtg2]])
    u_dist[2] = sum_rtg2
    
     # Rtg3
    sum_rtg3 = sum(u[protein_lookup[:Rtg3]])
    u_dist[11] = sum_rtg3
    
     # Rtg1
    sum_rtg1 = sum(u[protein_lookup[:Rtg1]])
    u_dist[14] = sum_rtg1
    
    # Mks
    sum_mks = sum(u[protein_lookup[:Mks]])
    u_dist[4] = sum_mks
    
    # Bmh 
    sum_bmh = sum(u[protein_lookup[:Bmh]])
    u_dist[6] = sum_bmh
    
    return u_dist
    
end

param = RetroSignalModel.rtgM4.param();
model = RetroSignalModel.rtgM4.model;
u = param.u; 
p = param.p;

In [ ]:
u_sep = redist2monomer(u,RetroSignalModel.get_protein_lookup(model))
for (j,i) in zip(u_sep, model.states)
    println(i,"  ",j)
end

In [ ]:
transRes = Try_conditions(u_sep,p, valid_setting(model))

In [ ]:
df = DataFrame(transRes)

In [ ]:
Rtg2 = sum(u[RetroSignalModel.get_protein_lookup(model)[:Rtg2]])

In [ ]:
RetroSignalModel.RTG_Response_Boolean

In [ ]:
TexPic = L"""
\documentclass[border={5pt 0pt 20pt 5pt}, preview]{standalone}

\usepackage{pgfplots}
\pgfplotsset{compat=1.15}
\usepackage{xcolor}


\usepackage{tikz}
\usepackage[utf8]{inputenc}
\usepackage{xfp}
\usepackage{booktabs}
\usetikzlibrary{calc}
\usepackage{graphicx}

\newcommand{\Del}[1]{\textit{$\Delta$\MakeLowercase{#1}}}
\newcommand\ratio[2]{%
    \pgfmathparse{ #1/(#1 + #2)}\pgfmathresult
}


\newcommand\colorBar{
    \begin{tikzpicture}[baseline={(0,0)}, inner sep=0,outer sep=0]

       
        \pgfdeclarehorizontalshading{someShading}{4cm}{
        color(0cm)=(green!0!gray);
        color(4cm)=(green!100!gray)
        }
        
        \shade [shading=someShading, yshift=-1cm]  (0,0) rectangle ++(1,0.3);
        \node [] at (0,-1.3) {0};
        \node [] at (1,-1.3) {1};
        
        \end{tikzpicture}
}


\newcommand\particle[2]{%
  \begin{tikzpicture}[x=2cm, y=2cm]
  

  % Calculate the ratio of nucleus to cytosol concentration
  \def\nratio{\fpeval{ round(#1/(#1 + #2),2)}  };
  \def\cratio{\fpeval{ round(#2/(#1 + #2),2)}  };

  \def\nconc{\fpeval{ round(#1,1)}  };
  \def\cconc{\fpeval{ round(#2,1)}  };
  
  %Node for Circles
  \node at (0,0) [circle,draw, scale=1 ,fill=green!\fpeval{round(100*(\cratio))}!gray] (c2) {};
  
  \node at (0,0) [circle,draw, scale=0.5 ,fill=green!\fpeval{round(100*(\nratio))}!gray] (c1) {};
  
  
  %Node for line ends
  \node [inner sep=0,outer sep=0, xshift=0.2cm] at (c1.east) (n_conc) {};
  \node [inner sep=0,outer sep=0,yshift = -0.09cm] at (n_conc.south) (c_conc) {};
  
  % Node for data
  \node [inner sep=0,outer sep=0, xshift=0.cm, text width = 0cm, align=left] at (n_conc.east) {\scalebox{.2}{\nconc} };
  \node [inner sep=0,outer sep=0, xshift=0.cm, text width = 0cm, align=left] at (c_conc.east) {\scalebox{.2}{\cconc}};
  
  % Draw Lines
  \draw[-, very thin] (n_conc) -- (c1);
  \draw[-, very thin] (c_conc) -- (0.16cm,-0.1065cm);
  
  \end{tikzpicture}
}

\newcommand\cell[2]{
    \begin{tikzpicture}[baseline={(c_conc.base)}, x=2cm, y=2cm, , outer sep=0]
  
        %Node for Circles
        \node at (0,0) [circle,draw, scale=1] (c2) {};
        
        \node at (0,0) [circle,draw, scale=0.5] (c1) {};
        
        
        %Node for line ends
        \node [inner sep=0,outer sep=0, xshift=0.2cm] at (c1.east) (n_conc) {};
        \node [inner sep=0,outer sep=0,yshift = -0.09cm] at (n_conc.south) (c_conc) {};
        
        % Node for data
        \node [inner sep=0,outer sep=0, xshift=0.cm, text width = 0cm, align=left] at (n_conc.east) {\scalebox{.2}{#1} };
        \node [inner sep=0,outer sep=0, xshift=0.cm, text width = 0cm, align=left] at (c_conc.east) {\scalebox{.2}{#2}};
        
        % Draw Lines
        \draw[-, very thin] (n_conc) -- (c1);
        \draw[-, very thin] (c_conc) -- (0.16cm,-0.1065cm);
        \end{tikzpicture}
    
}

\newcommand\drawcell[2]{%
    \begin{tikzpicture}[baseline={(c_conc.base)}, outer sep=0]
        \def\sc{3}
        \node at (0,0) [scale=\sc] {\cell{#1}{#2}};
    \end{tikzpicture}
}

\newcommand\drawRatio[2]{%
    \begin{tikzpicture}[baseline={(c_conc.base)}, outer sep=0]
        \def\sc{3}
        \node at (0,0) [scale=\sc] {\particle{#1}{#2}};
    \end{tikzpicture}
}



\begin{document}

\begin{tabular}[c]{p{1.4cm}p{1.8cm}p{2.8cm}|p{1.8cm}p{1.6cm}}
    \toprule
    \multicolumn{1}{c}{} & \multicolumn{2}{c}{\textbf{Healthy Mitochondria}} & \multicolumn{2}{c}{\textbf{Damaged Mitochondria}} \\ \midrule
     &\textbf{Simulation} & \multicolumn{1}{c}{\textbf{Data}}\hspace{0.52cm}  & \textbf{Simulation} & \hspace{0.5cm}\textbf{Data}  \\ 
     &   \multicolumn{4}{c}{\textbf{Rtg3-GFP}}  \\
    WT & \drawRatio{%$(transRes["nuc"][7])}{%$(transRes["cyt"][7])} & \drawRatio{0}{1} & \drawRatio{%$(transRes["nuc"][8])}{%$(transRes["cyt"][8])} & \drawRatio{1}{0} \\
    \Del{Rtg1} & \drawRatio{%$(transRes["nuc"][3])}{%$(transRes["cyt"][3])} & \drawRatio{1}{0}  & \drawRatio{%$(transRes["nuc"][4])}{%$(transRes["cyt"][4])} & \drawRatio{1}{0} \\
    \Del{Rtg2} & \drawRatio{%$(transRes["nuc"][5])}{%$(transRes["cyt"][5])} & \drawRatio{0}{1}& \drawRatio{%$(transRes["nuc"][6])}{%$(transRes["cyt"][6])} & \drawRatio{0}{1} \\
    \Del{Mks} & \drawRatio{%$(transRes["nuc"][18])}{%$(transRes["cyt"][18])} & \drawRatio{1}{0}& \multicolumn{2}{r}{}  \\
    \Del{Rtg2}\Del{Mks} & \drawRatio{%$(transRes["nuc"][17])}{%$(transRes["cyt"][3])} & \drawRatio{1}{0} & \multicolumn{2}{r}{} \\ 
     &   \multicolumn{4}{c}{\textbf{Rtg1-GFP}}   \\
    WT & \drawRatio{%$(transRes["nuc"][15])}{%$(transRes["cyt"][15])} & \drawRatio{0}{1} & \drawRatio{%$(transRes["nuc"][16])}{%$(transRes["cyt"][16])} & \drawRatio{1}{0} \\
    \Del{Rtg3} & \drawRatio{%$(transRes["nuc"][11])}{%$(transRes["cyt"][11])} & \drawRatio{0}{1}& \drawRatio{%$(transRes["nuc"][12])}{%$(transRes["cyt"][12])} & \drawRatio{0}{1} \\
    \Del{Rtg2} & \drawRatio{%$(transRes["nuc"][13])}{%$(transRes["cyt"][13])} & \drawRatio{0}{1} & \drawRatio{%$(transRes["nuc"][14])}{%$(transRes["cyt"][14])} & \drawRatio{0}{1} \\ 
    \Del{Mks} & \drawRatio{%$(transRes["nuc"][20])}{%$(transRes["cyt"][20])} & \drawRatio{1}{0}& \multicolumn{2}{r}{}  \\
    \Del{Rtg2}\Del{Mks} & \drawRatio{%$(transRes["nuc"][19])}{%$(transRes["cyt"][19])} & \drawRatio{1}{0} & \multicolumn{2}{r}{} \\ 
      &   \multicolumn{4}{c}{\textbf{Rtg2-GFP}}   \\
    WT & \drawRatio{0}{%$Rtg2} & \drawRatio{0}{1} & \drawRatio{0}{%$Rtg2} & \drawRatio{0}{1} \\
    &  \multicolumn{4}{c}{
        
    \begin{tikzpicture}
        \node [] (cell) {\drawcell{Nucleus}{Cytosol}};
        \node [ right of=cell, xshift=1.5cm] (col) {\colorBar};
    \end{tikzpicture}
    } 
    
    \\
    \bottomrule
\end{tabular}

\end{document}
"""



In [ ]:
open("SteadyStates/result/verification/PLOT_SteadyStateVerification.tex", "w") do io
           write(io, TexPic)
       end

# ;cd SteadyStates/result/verification/ 

In [ ]:
;xelatex PLOT_SteadyStateVerification  -synctex=1

In [ ]:
;cd ../../../

In [ ]:
;pwd